In [8]:
def load_dataset(txt_file):
    data, labels = [], []
    
    with open(txt_file, 'r') as file:
        for line in file.readlines():
            image_path = line.strip()
            full_path = os.path.join(image_path)
            if os.path.exists(full_path):
                label = image_path.split('/')[1] 
                data.append(full_path)
                labels.append(label)
    
    return pd.DataFrame({'image_path': data, 'label': labels})

In [ ]:
input_file = "train.txt"  # Replace with your actual filename
output_file = "train2.txt"  # The filtered output file

# Read the file and filter lines
with open(input_file, "r") as f:
    lines = f.readlines()

# Keep lines that contain "museum-indoor" or "museum-outdoor"
filtered_lines = [line.strip() for line in lines if "museum-indoor" in line or "museum-outdoor" in line]

# Write the filtered lines to the output file
with open(output_file, "w") as f:
    for line in filtered_lines:
        f.write(line + "\n")

print("Filtering complete. Check", output_file)

In [9]:
import os
import cv2
import numpy as np
import pandas as pd
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier

# ---- 1. Load Dataset ----

# ---- 2. Feature Extraction ----
def extract_features(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (64, 64))  # Resize for uniformity
    
    # Compute histograms for each color channel (Blue, Green, Red)
    hist_b = cv2.calcHist([image], [0], None, [256], [0, 256])
    hist_g = cv2.calcHist([image], [1], None, [256], [0, 256])
    hist_r = cv2.calcHist([image], [2], None, [256], [0, 256])

    # Normalize histograms
    hist_b = cv2.normalize(hist_b, hist_b).flatten()
    hist_g = cv2.normalize(hist_g, hist_g).flatten()
    hist_r = cv2.normalize(hist_r, hist_r).flatten()

    # Concatenate histograms into a single feature vector
    feature_vector = np.concatenate([hist_b, hist_g, hist_r])

    return feature_vector
# ---- 3. Load & Process Dataset ----
train_txt = "train.txt"  # Your txt file listing images

dataset = load_dataset(train_txt)

# Extract features for all images
features = np.array([extract_features(path) for path in dataset['image_path']])
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(dataset['label'])
print("Dataset Loaded:", dataset.shape)
print(dataset.head())
print("Features Shape:", features.shape)  # Should be (num_samples, num_features)
print("Labels Shape:", y_encoded.shape)  # Should be (num_samples,)




Dataset Loaded: (10000, 2)
                          image_path           label
0  train/museum-outdoor/00002789.jpg  museum-outdoor
1  train/museum-outdoor/00001215.jpg  museum-outdoor
2  train/museum-outdoor/00002469.jpg  museum-outdoor
3  train/museum-outdoor/00000891.jpg  museum-outdoor
4  train/museum-outdoor/00000515.jpg  museum-outdoor
Features Shape: (10000, 768)
Labels Shape: (10000,)


In [11]:
dt_model = DecisionTreeClassifier(max_depth=10)  # You can change max_depth
dt_model.fit(features, y_encoded)

# ---- 5. Save the Model ----
joblib.dump(dt_model, "decision_tree.pkl")
joblib.dump(encoder, "label_encoder.pkl")  # Save label encoder to decode predictions
print("Model saved as decision_tree.pkl")


Model saved as decision_tree.pkl


In [12]:
import joblib
import numpy as np
import pandas as pd
import cv2
import os
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# ---- Load Trained Model & Label Encoder ----
dt_model = joblib.load("decision_tree_model.pkl")
encoder = joblib.load("label_encoder.pkl")

# ---- Load Test Dataset ----
test_txt = "val.txt"  # Change to your test dataset file
test_dataset = load_dataset(test_txt)
print(test_dataset.head())

# Extract features
test_features = np.array([extract_features(path) for path in test_dataset['image_path']])
print(test_features)

# ---- Predict ----
predictions = dt_model.predict(test_features)
predicted_labels = encoder.inverse_transform(predictions)  # Convert back to original labels

# ---- Save Predictions ----
test_dataset['Predicted Label'] = predicted_labels
test_dataset.to_csv("test_predictions.csv", index=False)
print("Predictions saved in test_predictions.csv")

# ---- Compute Accuracy & Metrics ----
y_true = encoder.transform(test_dataset['label'])
accuracy = accuracy_score(y_true, predictions)
report = classification_report(y_true, predictions, target_names=encoder.classes_, digits=4)

print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:\n", report)


                                      image_path           label
0  val/museum-outdoor/Places365_val_00010130.jpg  museum-outdoor
1  val/museum-outdoor/Places365_val_00001310.jpg  museum-outdoor
2  val/museum-outdoor/Places365_val_00004772.jpg  museum-outdoor
3  val/museum-outdoor/Places365_val_00019233.jpg  museum-outdoor
4  val/museum-outdoor/Places365_val_00010906.jpg  museum-outdoor
[[0.16869023 0.20072003 0.3202979  ... 0.02421181 0.01100537 0.04402148]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.01350957 0.02161531 0.01891339]
 ...
 [0.36373875 0.27965844 0.27051926 ... 0.0027731  0.0027731  0.00554619]
 [0.         0.         0.         ... 0.02896342 0.02633038 0.01579823]
 [0.00734231 0.01560241 0.0247803  ... 0.         0.         0.00146194]]
Predictions saved in test_predictions.csv
Accuracy: 0.7550
Classification Report:
                 precision    recall  f1-score   support

 museum-indoor     0.7476 